In [ ]:
#DOCUMENTATION - https://docs.trackstarhq.com/introduction
import requests
import pandas as pd
import os
from pathlib import Path
# set openai api key
from dotenv import load_dotenv
load_dotenv()
TRACKSTAR_API_KEY = os.getenv("TRACKSTAR_API_KEY")

# Files will be saved in the current directory
print(f"CSV files will be saved in: {Path.cwd()}")
#Get this information from the web portal 
customer_id = "shipcalm_lookoptic"
connection_id = "ab896808e6d1495b82dbfdf9f8435572"
integration_name = "Infoplus"
access_token = "trackstar_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uX2lkIjoiYWI4OTY4MDhlNmQxNDk1YjgyZGJmZGY5Zjg0MzU1NzIiLCJpbnRlZ3JhdGlvbl9uYW1lIjoiaW5mb3BsdXMifQ.75BBuw4TwKaccpXN_xWrOCyO1PkkwlTGgBk1LLv7UIA"
#Get available actions for the integration
url = "https://production.trackstarhq.com/wms/integrations/"+integration_name.lower()
headers = {"x-trackstar-api-key": TRACKSTAR_API_KEY}
response = requests.request("GET", url, headers=headers)
#response.json()['data'][0]['available_actions']
def paginate_endpoint(endpoint,page_token):
    print(page_token)
    url = "https://production.trackstarhq.com/wms/"+endpoint
    headers = {
            "x-trackstar-api-key": TRACKSTAR_API_KEY,
            "x-trackstar-access-token": access_token
        }

    if page_token:
        querystring = {"page_token":page_token}
        response = requests.request("GET", url, headers=headers,params=querystring)
    else:
        response = requests.request("GET", url, headers=headers)
    
    return response
def get_data(endpoint):
    output = []
    response = paginate_endpoint(endpoint,"")
    output = output + response.json()['data']
    page_token = response.json()['next_token']
    while page_token:
        response = paginate_endpoint(endpoint,page_token)
        output = output + response.json()['data']
        page_token = response.json()['next_token']
    return output

In [ ]:
###########################
# Inventory
############################
output = get_data("inventory")
for row in output:
    row["substitute_skus"]           = str(row["substitute_skus"])
    row["inventory_by_warehouse_id"] = str(row["inventory_by_warehouse_id"])
    row["lots"]                      = str(row["lots"])
    row["measurements"]              = str(row["measurements"])
    row["locations"]                 = str(row["locations"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","name","sku","active","awaiting","onhand","committed","unfulfillable","fulfillable","unsellable","sellable","substitute_skus","inventory_by_warehouse_id","lots","measurements","locations","external_system_url"]) 
df.to_csv("inventory.csv")   
###########################
# Products
############################
output = get_data("products")
for row in output:
    row["inventory_items"]   = str(row["inventory_items"])
    row["supplier_products"] = str(row["supplier_products"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","name","sku","gtin","inventory_items","is_kit","active","supplier","country_of_origin","harmonized_code","supplier_products","external_system_url"]) 
df.to_csv("products.csv")
############################
# Shipping Methods
############################
output = get_data("shipping-methods")
df = pd.DataFrame.from_records(output,index=['id', 'created_date','updated_date','name','carrier']) 
df.to_csv("shipping_methods.csv")
############################
# Shipment Suppliers
############################
output = get_data("inbound-shipments/suppliers")
df = pd.DataFrame.from_records(output,index=['id', 'created_date','updated_date']) 
df.to_csv("suppliers.csv")
############################
# Inbound Shipments
############################
output = get_data("inbound-shipments")
for row in output:
    row["line_items"]         = str(row["line_items"])
    row["receipts"]           = str(row["receipts"])
    row["ship_from_address"]  = str(row["ship_from_address"])
    row["tracking_numbers"]   = str(row["tracking_numbers"])
    row["notes"]              = str(row["notes"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","purchase_order_number","status","raw_status","supplier","expected_arrival_date","warehouse_id","line_items","receipts","ship_from_address","tracking_numbers","notes","external_system_url"]) 
df.to_csv("inbound_shipments.csv")
############################
# Order Channels
############################
output = get_data("orders/channels")
#for row in output:
#    row["line_items"]         = str(row["line_items"])
#    row["receipts"]           = str(row["receipts"])
#    row["tracking_numbers"]   = str(row["tracking_numbers"])
#    row["notes"]              = str(row["notes"])
#df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","purchase_order_number","status","raw_status","supplier","expected_arrival_date","warehouse_id","line_items","receipts","ship_from_address","tracking_numbers","notes","external_system_url"]) 
#df.to_csv("/Users/danieljacobini/Downloads/inbound_shipments.csv")
############################
# Orders
############################
output = get_data("orders")
line_items = []
for row in output:
    line_items = line_items + [dict(item, **{'id':row["id"]}) for item in row["line_items"]]
df = pd.DataFrame.from_records(line_items,index=["product_id","sku","quantity","unit_price","is_picked","discount_amount","id"])
df.to_csv("orders_line_items.csv")
for row in output:
    row["ship_to_address"] = str(row["ship_to_address"])
    row["line_items"]      = str(row["line_items"])
    row["tags"]            = str(row["tags"])
    row["shipments"]       = str(row["shipments"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","reference_id","order_number","status","raw_status","channel","type","trading_partner","shipping_method","invoice_currency_code","total_price","total_tax","total_discount","total_shipping","ship_to_address","line_items","tags","required_ship_date","shipments","external_system_url"]) 
df.to_csv("orders.csv")

# takes about 20 minutes to run

In [ ]:
import numpy as np
import ast
from datetime import datetime, timedelta, date
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

print("=== INVENTORY TIME SERIES ANALYSIS SYSTEM ===")
print("Loading datasets...")

# Load Datasets with Date Parsing 
inbound_shipments = pd.read_csv("inbound_shipments.csv", 
                                parse_dates=['created_date', 'updated_date', 'expected_arrival_date'])

inventory = pd.read_csv("inventory.csv", 
                        parse_dates=['created_date', 'updated_date'])

orders_line_items = pd.read_csv("orders_line_items.csv")

orders = pd.read_csv("orders.csv", 
                     parse_dates=['created_date', 'updated_date', 'required_ship_date'])

products = pd.read_csv("products.csv", 
                       parse_dates=['created_date', 'updated_date'])

shipping_methods = pd.read_csv("shipping_methods.csv", 
                               parse_dates=['created_date', 'updated_date'])

suppliers = pd.read_csv("suppliers.csv")

# Keep datetime columns as date objects (not converting to string)
inbound_shipments['created_date'] = inbound_shipments['created_date'].dt.date
inbound_shipments['updated_date'] = inbound_shipments['updated_date'].dt.date
inbound_shipments['expected_arrival_date'] = inbound_shipments['expected_arrival_date'].dt.date

inventory['created_date'] = inventory['created_date'].dt.date
inventory['updated_date'] = inventory['updated_date'].dt.date

orders['created_date'] = orders['created_date'].dt.date
orders['updated_date'] = orders['updated_date'].dt.date
orders['required_ship_date'] = orders['required_ship_date'].dt.date

products['created_date'] = products['created_date'].dt.date
products['updated_date'] = products['updated_date'].dt.date

shipping_methods['created_date'] = shipping_methods['created_date'].dt.date
shipping_methods['updated_date'] = shipping_methods['updated_date'].dt.date

print("✓ Datasets loaded successfully")

# Helper Functions
def inspect_dataset(df):
    print(f"\n{df} Dataset Overview:")
    print("\nHEAD:\n", df.head())
    print("\nINFO:\n")
    print(df.info())
    print("\nMISSING VALUES:\n", df.isnull().sum())
    print("\n" + "-"*80 + "\n")
    
def safe_eval_list(item_list):
    """Safely evaluate string representations of lists"""
    if isinstance(item_list, str):
        try:
            return ast.literal_eval(item_list)
        except (ValueError, SyntaxError):
            return []
    elif isinstance(item_list, list):
        return item_list
    else:
        return []

def extract_unit_quantity(item_list):
    """Extract unit_quantity from inventory_items"""
    if isinstance(item_list, str):  # If it's stored as a string, convert it to a list
        try:
            item_list = ast.literal_eval(item_list)
        except (ValueError, SyntaxError):
            return 0  # Return 0 if conversion fails
    if isinstance(item_list, list) and len(item_list) > 0:
        return item_list[0].get('unit_quantity', 0)  # Get unit_quantity from first item
    return 0  # Return 0 if no valid data

def parse_date(date_value):
    """Parse date value to date object (date only, no time)"""
    if pd.isna(date_value):
        return None
    try:
        if isinstance(date_value, str):
            dt = datetime.fromisoformat(date_value.replace("Z", "+00:00"))
            return dt.date()
        elif isinstance(date_value, datetime):
            return date_value.date()
        elif isinstance(date_value, date):
            return date_value
        else:
            return None
    except:
        return None

# Extract unit_quantity from products
print("Extracting unit_quantity from inventory_items...")
products['unit_quantity'] = products['inventory_items'].apply(extract_unit_quantity)
print(f"✓ Unit quantity extracted for {len(products)} products")
print("Sample unit_quantity values:", products['unit_quantity'].head())

def build_sales_time_series():
    """Build time series for sales data including returns (negative quantities)"""
    print("Building sales time series...")
    
    # Merge orders with line items to get dates
    sales_data = orders_line_items.merge(
        orders[['id', 'created_date']], 
        left_on='id', 
        right_on='id', 
        how='left'
    ).rename(columns={'created_date': 'order_date'})
    
    # Group sales by SKU and create time series
    sales_time_series = {}
    
    for sku in sales_data['sku'].unique():
        sku_sales = sales_data[sales_data['sku'] == sku].copy()
        
        # Group by date and aggregate quantities for same-day transactions
        daily_sales = defaultdict(lambda: {'quantity': 0, 'total_value': 0, 'unit_prices': []})
        
        for _, row in sku_sales.iterrows():
            order_date = parse_date(row['order_date'])
            if order_date:
                daily_sales[order_date]['quantity'] += row['quantity']
                unit_price = row.get('unit_price', 0)
                daily_sales[order_date]['total_value'] += row['quantity'] * unit_price
                daily_sales[order_date]['unit_prices'].append(unit_price)
        # Convert to time series format
        time_series = []
        for order_date, data in daily_sales.items():
            avg_unit_price = data['total_value'] / data['quantity'] if data['quantity'] != 0 else 0
            time_series.append({
                'date': order_date,  # Keep as date object
                'quantity': data['quantity'],
                'unit_price': avg_unit_price,
                'total_value': data['total_value']
            })
        
        # Sort by date
        time_series.sort(key=lambda x: x['date'])
        sales_time_series[sku] = time_series
    
    return sales_time_series

def build_received_time_series():
    """Build time series for received inventory from inbound shipments"""
    print("Building received inventory time series...")
    
    received_time_series = {}
    
    # Group by SKU and date first
    daily_receipts = defaultdict(lambda: defaultdict(lambda: {'quantity': 0, 'shipment_ids': [], 'purchase_orders': []}))
    
    for _, shipment in inbound_shipments.iterrows():
        line_items = safe_eval_list(shipment['line_items'])
        shipment_date = parse_date(shipment['created_date'])
        
        if not shipment_date or not line_items:
            continue
            
        for item in line_items:
            sku = item.get('sku')
            quantity = item.get('quantity', item.get('received_quantity', 0))
            
            if sku and quantity:
                daily_receipts[sku][shipment_date]['quantity'] += quantity
                daily_receipts[sku][shipment_date]['shipment_ids'].append(shipment['id'])
                daily_receipts[sku][shipment_date]['purchase_orders'].append(shipment.get('purchase_order_number', 'Unknown'))
    
    # Convert to time series format
    for sku, date_data in daily_receipts.items():
        time_series = []
        for receipt_date, data in date_data.items():
            time_series.append({
                'date': receipt_date,  # Keep as date object
                'quantity': data['quantity'],
                'shipment_ids': list(set(data['shipment_ids'])),
                'purchase_orders': list(set(data['purchase_orders']))
            })
        
        # Sort by date
        time_series.sort(key=lambda x: x['date'])
        received_time_series[sku] = time_series
    
    return received_time_series

def build_historical_inventory_corrected(sku, sales_ts, received_ts):
    """Calculate historical inventory using inventory.sellable as current inventory - CORRECTED VERSION"""
    
    # Get current inventory from inventory table's 'sellable' column
    inventory_row = inventory[inventory['sku'] == sku]
    if inventory_row.empty:
        current_inventory = 0
    else:
        current_inventory = inventory_row.iloc[0]['sellable']
    
    # Find all transaction dates
    all_dates = set()
    if sku in sales_ts:
        all_dates.update([entry['date'] for entry in sales_ts[sku]])
    if sku in received_ts:
        all_dates.update([entry['date'] for entry in received_ts[sku]])
    
    if not all_dates:
        return []
    
    # Group transactions by date
    daily_transactions = defaultdict(lambda: {'sales': 0, 'receipts': 0})
    
    if sku in sales_ts:
        for sale in sales_ts[sku]:
            sale_date = sale['date']
            daily_transactions[sale_date]['sales'] += sale['quantity']
    
    if sku in received_ts:
        for receipt in received_ts[sku]:
            receipt_date = receipt['date']
            daily_transactions[receipt_date]['receipts'] += receipt['quantity']
    
    # Create sorted list of dates (most recent first for backwards calculation)
    sorted_dates = sorted(all_dates, reverse=True)
    
    # Calculate historical inventory by working backwards from current
    historical_inventory = []
    inventory_after_date = current_inventory
    
    # Work backwards through dates
    for transaction_date in sorted_dates:
        daily_data = daily_transactions[transaction_date]
        net_change = daily_data['receipts'] - daily_data['sales']
        
        # Record the inventory level AFTER this date's transactions
        historical_inventory.append({
            'date': transaction_date,  # Keep as date object
            'inventory': int(inventory_after_date),
            'sales': int(daily_data['sales']),
            'receipts': int(daily_data['receipts']),
            'net_change': int(net_change)
        })
        
        # Calculate what inventory was BEFORE this date's transactions (for next iteration)
        inventory_after_date = inventory_after_date - net_change
    
    # Sort chronologically (oldest first)
    historical_inventory.sort(key=lambda x: x['date'])
    
    return historical_inventory


def create_corrected_sku_dataframe():
    """Create dataframe with corrected inventory and requested columns only"""
    print("Creating corrected SKU dataframe...")
    
    # Start with products table for SKU, name, created_date, and unit_price
    base_data = products[['sku', 'name', 'created_date', 'unit_price']].copy()
    
    # Add updated_date from inventory table
    inventory_updates = inventory[['sku', 'updated_date']].rename(columns={'updated_date': 'updated_date'})
    base_data = base_data.merge(inventory_updates, on='sku', how='left')
    
    # Initialize time series columns
    base_data['historical_inventory'] = None
    base_data['historical_sales'] = None
    base_data['historical_received'] = None
    
    print(f"Processing {len(base_data)} SKUs...")
    
    # Process each SKU
    for idx, row in base_data.iterrows():
        sku = row['sku']
        
        # Build corrected time series for this SKU
        hist_inventory = build_historical_inventory_corrected(sku, sales_ts, received_ts)
        hist_sales = sales_ts.get(sku, [])
        hist_received = received_ts.get(sku, [])
        
        # Store time series
        base_data.at[idx, 'historical_inventory'] = hist_inventory
        base_data.at[idx, 'historical_sales'] = hist_sales
        base_data.at[idx, 'historical_received'] = hist_received
    
    return base_data


# Execute the corrected analysis
print("\n=== EXECUTING CORRECTED ANALYSIS ===")

# Build time series (sales and received remain the same as they were working correctly)
sales_ts = build_sales_time_series()
print(f"✓ Sales time series built for {len(sales_ts)} SKUs")

received_ts = build_received_time_series()
print(f"✓ Received inventory time series built for {len(received_ts)} SKUs")

# Create corrected comprehensive dataframe
sku_timeseries_df_corrected = create_corrected_sku_dataframe()
print(f"✓ Corrected dataframe created with {len(sku_timeseries_df_corrected)} SKUs")

# Display sample of corrected data
print("\nSample of corrected data:")
print(sku_timeseries_df_corrected.head())

In [ ]:
df = sku_timeseries_df_corrected
inspect_dataset(df)
df

In [ ]:
df.to_parquet('sku_data.parquet', index=False)

In [ ]:
inspect_dataset(df)

In [ ]:
# Print all SKUs
for idx, row in df.iterrows():
    sku = row['sku']
    print(f"Row for SKU {sku}:")
    for col, value in row.items():
        print(f"  {col}: {value}")
    print("-" * 40)

In [ ]:
# Find SKUs with negative quantities in historical_sales
negative_quantity_skus = []

for idx, row in df.iterrows():
    sku = row['sku']
    historical_sales = row['historical_sales']
    
    # Check each sale in historical_sales
    for sale in historical_sales:
        if sale['quantity'] < 0:
            negative_quantity_skus.append({
                'sku': sku,
                'date': sale['date'],
                'quantity': sale['quantity']
            })

# Display results
if negative_quantity_skus:
    print("\nFound SKUs with negative quantities:")
    for item in negative_quantity_skus:
        print(f"SKU: {item['sku']}, Date: {item['date']}, Quantity: {item['quantity']}")
else:
    print("\nNo SKUs found with negative quantities")
